# Table of Contents

## 1. Data Security Implications
## 2. Regional segmentation of data
## 3. Creating an Exclusion Flag for Low-Activity Customers (Customers with Less Than 5 Orders)
## 4. Creating a Profiling Variable Based on Age, Income, and Number of Dependents
## 5. Importing and Merging Department Data
## 6. Exporting Data

# Importing Libraries and Data

In [2]:
import pandas as pd
import numpy as np
import os

path = r'C:\Users\Charles Yi\04-2023 Instacart Basket Analysis\02 Data\Prepared Data'

df = pd.read_pickle(os.path.join(path, 'cust_ords_prods_prior_2.pkl'))

# 1. Data Security Implications

In [3]:
# df dtypes

df.dtypes

order_id                           int64
user_id                           object
order_number                       int64
orders_day_of_week                 int64
order_hour_of_day                  int64
days_since_prior_order           float64
product_id                         int64
add_to_cart_order                  int64
reordered                          int64
product_name                      object
aisle_id                           int64
department_id                      int64
prices                           float64
price_range                       object
busiest_day                       object
busiest_period_of_day             object
order_attribute                   object
max_order                          int64
loyalty_flag                      object
mean_price                       float64
spending_class                    object
median_order_interval            float64
frequent_customer                 object
first_name                        object
last_name       

## The df contains several PII, including name, state of residence, and age. Such information should be handled, stored, and discarded in a secure manner and according to the regulations and laws that govern PII to protect the identities of the customers.

# 2. Regional segmentation of data

## 2a. Number of Customers Per Region

In [4]:
# creating for-loop function to categorize states by region

results = []

for value in df['state']:
    if value in ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']:
        results.append('Northeast')
    elif value in ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']:
        results.append('Midwest')
    elif value in ['Delaware', 'Maryland','District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']:
        results.append('South')
    elif value in ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']:
        results.append('West')
    else:
        results.append(np.nan)


In [5]:
# creating region column with results

df['region'] = results

In [6]:
# counting values in region column to check work

df['region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

In [7]:
# eliminating column restrictions

pd.options.display.max_columns = None

In [8]:
# counting spending class by region and spending flag

customers_region = df.groupby(['spending_class_updated', 'region']).agg({'spending_class_updated' : 'count'}).rename(columns = {'spending_class_updated' : 'number_of_customers'})

customers_region.sort_values(by = 'number_of_customers', ascending = False)

number_of_customers
spending_class_updated region                        
Low spender            South                 10582194
                       West                   8132559
                       Midwest                7441350
                       Northeast              5614511
High spender           South                   209691
                       West                    160354
                       Midwest                 155975
                       Northeast               108225

In [9]:
# same result using pd.crosstab function

pd.crosstab(df['region'], df['spending_class_updated'], dropna = False)

spending_class_updated,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


## The South has both the higest number of low and high spenders, and hence, the largest customer base. The Northeast has the lowest number of low and high spenders. It is also Instacart's smallest regional market.

## 2b. Percentage of High Spending Customers Per Region

In [10]:
# creating seperate df with binary indicator for high spenders

df_high_spenders = df[['region', 'spending_class_updated']]

df_high_spenders.loc[df_high_spenders['spending_class_updated'] == 'High spender', 'high_spender_?'] = 1

df_high_spenders.loc[df_high_spenders['spending_class_updated'] == 'Low spender', 'high_spender_?'] = 0

df_high_spenders.head()

C:\Users\Charles Yi\AppData\Local\Temp\ipykernel_34460\3052969396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_spenders.loc[df_high_spenders['spending_class_updated'] == 'High spender', 'high_spender_?'] = 1


,region,spending_class_updated,high_spender_?
0,South,Low spender,0.0
1,South,Low spender,0.0
2,South,Low spender,0.0
3,South,Low spender,0.0
4,South,Low spender,0.0


In [11]:
# transforming df via lambda to add percentage of high spenders in each region

df_high_spenders['percentage_high_spenders'] = df_high_spenders.groupby('region')['high_spender_?'].transform(lambda x: (x.sum() / x.count()) * 100) 

C:\Users\Charles Yi\AppData\Local\Temp\ipykernel_34460\3263006038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_spenders['percentage_high_spenders'] = df_high_spenders.groupby('region')['high_spender_?'].transform(lambda x: (x.sum() / x.count()) * 100)


In [12]:
# checking result

df_high_spenders.head(100)

,region,spending_class_updated,high_spender_?,percentage_high_spenders
0,South,Low spender,0.0,1.943043
1,South,Low spender,0.0,1.943043
2,South,Low spender,0.0,1.943043
3,South,Low spender,0.0,1.943043
4,South,Low spender,0.0,1.943043
...,...,...,...,...
95,Midwest,Low spender,0.0,2.053025
96,Midwest,Low spender,0.0,2.053025
97,Midwest,Low spender,0.0,2.053025
98,Midwest,Low spender,0.0,2.053025


In [13]:
# viewing percentage of high spenders by region

percentage_high_spenders = df_high_spenders.groupby('region').agg({'percentage_high_spenders' : 'mean'})

percentage_high_spenders

,percentage_high_spenders
region,
Midwest,2.053025
Northeast,1.891141
South,1.943043
West,1.933627


In [14]:
# sorting percentage_high_spenders and rounding to two decimal places

percentage_high_spenders.sort_values(by = ('percentage_high_spenders'), ascending = False).round(2)

,percentage_high_spenders
region,
Midwest,2.05
South,1.94
West,1.93
Northeast,1.89


## Overall, the percentage of high spenders in each region is low. Interestingly, however, the Midwest has the highest percentage of high spenders, despite having the second-lowest number of high spenders among all regions.

# 3. Creating an Exclusion Flag for Low-Activity Customers (Customers with Less Than 5 Orders)

In [15]:
# creating binary flag for customers with less than 5 max orders

df.loc[df['max_order'] < 5, 'low_activity_customer_?'] = 1

df.loc[df['max_order'] >= 5, 'low_activity_customer_?'] = 0

In [16]:
# checking results

pd.options.display.max_rows = None

df[['max_order', 'low_activity_customer_?']][:100]

,max_order,low_activity_customer_?
0,10,0.0
1,10,0.0
2,10,0.0
3,10,0.0
4,10,0.0
5,10,0.0
6,10,0.0
7,10,0.0
8,10,0.0
9,10,0.0


In [17]:
# excluding low activity customers from df

df_2 = df.loc[df['low_activity_customer_?'] == 0]

In [18]:
# checking results

df_2['low_activity_customer_?'].value_counts(dropna = False)

0.0    30964564
Name: low_activity_customer_?, dtype: int64

## 30,964,564 customers have more than 5 orders. A seperate df, df_2, was created with only high_activity_customers included.

# 4. Creating a Profiling Variable Based on Age, Income, and Number of Dependents

In [19]:
# creating separate df with relevant columns

df_3 = df[['user_id', 'region', 'Age', 'income', 'n_dependants', 'department_id', 'product_name', 'prices', 'add_to_cart_order', 'orders_day_of_week', 'order_hour_of_day', 'days_since_prior_order']]

df_3.head()

,user_id,region,Age,income,n_dependants,department_id,product_name,prices,add_to_cart_order,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,1,South,31,40423,3,7,Soda,9.0,1,2,8,NaN
1,1,South,31,40423,3,7,Soda,9.0,1,3,7,15.0
2,1,South,31,40423,3,7,Soda,9.0,1,3,12,21.0
3,1,South,31,40423,3,7,Soda,9.0,1,4,7,29.0
4,1,South,31,40423,3,7,Soda,9.0,1,4,15,28.0


## 4a. Grouping Customers by Age

In [20]:
# overview of age statistics

df_3['Age'].describe()

count    3.240486e+07
mean     4.946527e+01
std      1.848558e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: Age, dtype: float64

In [21]:
# grouping customers by age

age_groups = []

for value in df['Age']:
    if value <= 44:
        age_groups.append('18-44')
    elif value > 44 and value <=64:
        age_groups.append('45-64')
    elif value >= 65:
        age_groups.append('65+')

In [22]:
# adding age_groups to df

df_3['age_group'] = age_groups

C:\Users\Charles Yi\AppData\Local\Temp\ipykernel_34460\1974202569.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['age_group'] = age_groups


## 4b. Grouping Customers by Income

In [27]:
# grouping customers based on income

income_group = []

for value in df_3['income']:
    if value >= 25000 & value <= 74999:
        income_group.append('Low income')
    elif value >= 75000 & value <= 149999:
        income_group.append('Middle income')
    elif value >= 150000:
        income_group.append('High income')

In [28]:
# adding income_group to df

df_3['income_group'] = income_group

C:\Users\Charles Yi\AppData\Local\Temp\ipykernel_34460\172519215.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['income_group'] = income_group


## 4c. Grouping Customers by Family Status

In [29]:
# overview of dependent stats

df_3['n_dependants'].describe()

count    3.240486e+07
mean     1.501896e+00
std      1.118865e+00
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      3.000000e+00
max      3.000000e+00
Name: n_dependants, dtype: float64

In [30]:
# grouping customers based on family status

family_group = []

for value in df_3['n_dependants']:
    if value == 0:
        family_group.append('Single')
    elif value > 0:
        family_group.append('Family')

In [31]:
# pairing df with family_group

df_3['family_group'] = family_group

C:\Users\Charles Yi\AppData\Local\Temp\ipykernel_34460\2732776551.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['family_group'] = family_group


# 5. Importing and Merging Department Data

In [32]:
# importing department df

dp = pd.read_csv(os.path.join(path, 'departments_wrangled.csv'), index_col = False)

In [33]:
# checking dp

dp

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [34]:
# renaming first column to department_id

dp.rename(columns = {'Unnamed: 0' : 'department_id'}, inplace = True)

In [35]:
# merging department df with df_3

df_4 = df_3.merge(dp, on = 'department_id')

In [36]:
# viewing result

df_4.head()

,user_id,region,Age,income,n_dependants,department_id,product_name,prices,add_to_cart_order,orders_day_of_week,order_hour_of_day,days_since_prior_order,age_group,income_group,family_group,department
0,1,South,31,40423,3,7,Soda,9.0,1,2,8,NaN,18-44,Low income,Family,beverages
1,1,South,31,40423,3,7,Soda,9.0,1,3,7,15.0,18-44,Low income,Family,beverages
2,1,South,31,40423,3,7,Soda,9.0,1,3,12,21.0,18-44,Low income,Family,beverages
3,1,South,31,40423,3,7,Soda,9.0,1,4,7,29.0,18-44,Low income,Family,beverages
4,1,South,31,40423,3,7,Soda,9.0,1,4,15,28.0,18-44,Low income,Family,beverages


# 6. Exporting Data

In [37]:
# exporting df as cust_ords_prods_prior_3.pkl

# df.to_pickle(os.path.join(path, 'cust_ords_prods_prior_3.pkl'))

# exporting df_2 as high_spending_customers.pkl

# df_2.to_pickle(os.path.join(path, 'high_spending_customers.pkl'))

# exporting df_4 as dep_cust_profile.pkl

# df_4.to_pickle(os.path.join(path, 'dep_cust_profile.pkl'))